# Bloomberg BQuant Spotlight Webinar Series: Consistent Dividend Growth
This is a companion notebook to the Consistent Dividend Growth webinar.

In [ ]:
import bql
import pandas as pd
import bqviz

In [ ]:
bq = bql.Service()

### Explore the CASH_DIVS field for one company

In [ ]:
# We will analyze Broadcom in this section
divs_10yr = bq.data.cash_divs(dates = bq.func.range('-10Y','0D'),dividend_type='regular',ca_date_type='effective_date').dropna()
divs_10yr_chgs = bq.func.diff(divs_10yr).dropna()

items = {
    '10_yr_div': divs_10yr,
    '10_yr_div_chg': divs_10yr_chgs
}

univ = bq.univ.list('AVGO UW Equity')

req = bql.Request(univ, items)

In [ ]:
resp = bq.execute(req)
combined = bql.combined_df(resp).set_index(['DATE']).dropna()
combined.head()

In [ ]:
bqviz.BarPlot(combined[['10_yr_div']]).set_style().show()

In [ ]:
bqviz.OverUnderPlot(combined[['10_yr_div_chg']]).set_style().show()

### Analyze Div Changes for a broader universe

In [ ]:
# We explore the dividends data item
index = 'SPX Index'
univ = bq.univ.members('SPX Index')

min_div_chg = bq.func.min(divs_10yr_chgs)

items = {
    'min_div_chg': min_div_chg
}

req = bql.Request(univ, items)

In [ ]:
resp = bq.execute(req)
combined = bql.combined_df(resp)
combined.sort_values('min_div_chg').head(10)

### Define screening criteria

In [ ]:
#check that the company pays dividends
div_yield = bq.data.dividend_yield()*100
pays_div = div_yield > 0

#check if constantily is paying dividends
has_cut_divs =  bq.func.if_(
    min_div_chg<0, True, False
)

#check if company is large enough to be considered
market_cap = bq.data.cur_mkt_cap(currency='USD')
large_enough = market_cap>'1B'


In [ ]:
#filtering the universe
univ_filtered = bq.univ.filter(univ,bq.func.and_(bq.func.and_(pays_div, has_cut_divs==False), large_enough))

items = {
    'div_yield': div_yield['value'],
    'mkt_cap': market_cap['value']
}

req = bql.Request(univ_filtered, items)

In [ ]:
resp = bq.execute(req)
combined = bql.combined_df(resp)
combined.head()

### Create a hypothetical portfolio weighted by the dividend yield

In [ ]:
weighting_metric = div_yield
weight = weighting_metric / bq.func.group(weighting_metric).sum().ungroup()

items = {
    'weight': weight['value'],
    'div_yield': div_yield['value'],
    'mkt_cap': market_cap['value']
}

req = bql.Request(univ_filtered, items)
resp = bq.execute(req)

In [ ]:
resp = bq.execute(req)
combined = bql.combined_df(resp)
combined.head()

In [ ]:
combined['weight'].sum()

### Compare Div Growth strategy to benchmark

In [ ]:
debt_to_cap = bq.data.lt_debt_to_tot_cap()
fcf_yield = bq.data.free_cash_flow_yield()

compare_items = {
    'div_yield': div_yield['value'],
    'market_cap': market_cap['value'],
    'debt_to_cap': debt_to_cap['value'],
    'fcf_yield': fcf_yield['value']
}


In [ ]:
def wgt_avg(bql_item, weighting_metric):
    bql_item_replacena = bq.func.replacenonnumeric(bql_item['value'], 0)
    wavg_item = bq.func.wavg(bq.func.group(bql_item_replacena['value']), bq.func.group(weighting_metric['value']))
    return wavg_item['value']

def get_wgt_avg_items(orig_dict, wgt_item):
    new_dict = {}
    for key, val in orig_dict.items():
        new_dict[key] = wgt_avg(val, wgt_item)
    return new_dict

In [ ]:
#div growth portfolio
dg_items = get_wgt_avg_items(compare_items, div_yield)
dg_req = bql.Request(univ_filtered, dg_items)

#benchmark
bmrk_items = get_wgt_avg_items(compare_items, market_cap)
bmrk_req = bql.Request(univ, bmrk_items)

In [ ]:
resp = bq.execute(dg_req)
div_growth_df = bql.combined_df(resp)
div_growth_df.index.values[0] = 'Div Growth'

In [ ]:
resp = bq.execute(bmrk_req)
bmrk_df = bql.combined_df(resp)
bmrk_df.index.values[0] = "Benchmark"

In [ ]:
comparison_df = pd.concat([div_growth_df, bmrk_df])
comparison_df

In [ ]:
plots = [
    (bqviz.BarPlot, 'Debt / Cap', ['debt_to_cap']),
    (bqviz.BarPlot, 'FCF yield', ['fcf_yield']),
    (bqviz.BarPlot, 'Div Yield', ['div_yield']),
    (bqviz.BarPlot, 'Mkt Cap', ['market_cap']),
]

comparison = bqviz.MultiGridPlot(comparison_df, plots, cols=2)

comparison.set_style().show()